In [ ]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model

In [ ]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

In [ ]:
# training script
source_dir = "train/bert"
script_name = "train_bert.py"

# environment file
environment_file = os.path.join("train", "bert", "train_bert_env.yml")

# azure ml settings
environment_name = "pl-env-lang"
experiment_name = "bert-livedoor"
compute_name = "gpu-instance01"

In [ ]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04"
)

In [ ]:
# 学習設定
src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=["--batch_size", 128, "--max_epochs", 10, "--gpus", 1],
    compute_target=compute_name,
    environment=env,
)

In [ ]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

In [ ]:
# モデル登録
run.register_model(
    model_name="bert-livedoor-model",
    model_path=os.path.join('outputs', 'model.onnx'),
    model_framework=Model.Framework.ONNX,
)